In [5]:
%load_ext watermark
%watermark -a "Sebastian Martins" -u -d -v -p numpy,pandas,matplotlib,sklearn

Sebastian Martins 
last updated: 2018-06-04 

CPython 3.6.3
IPython 6.1.0

numpy 1.13.3
pandas 0.20.3
matplotlib 2.1.0
sklearn 0.19.1


In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("1 - FC fast ai - df total vs pred.xls")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 21 columns):
Fecha                    846 non-null datetime64[ns]
PedidosReales            846 non-null int64
SolerDiaPred             846 non-null int64
SolerNochePred           846 non-null int64
TotalesPred              846 non-null int64
difRealPred              846 non-null int64
difAbsRealPred           846 non-null int64
FechaYear                846 non-null int64
FechaMonth               846 non-null int64
FechaWeek                846 non-null int64
FechaDay                 846 non-null int64
FechaDayofweek           846 non-null int64
FechaDayofyear           846 non-null int64
FechaIs_month_end        846 non-null int64
FechaIs_month_start      846 non-null int64
FechaIs_quarter_end      846 non-null int64
FechaIs_quarter_start    846 non-null int64
FechaIs_year_end         846 non-null int64
FechaIs_year_start       846 non-null int64
FechaElapsed             846 non-null int64
Pedido

In [2]:
#agregando la cantidad de ventas del día previo
ventaAyer = df[["Fecha", "PedidosReales"]].copy()
ventaAyer["PedidosReales"] = ventaAyer['PedidosReales'].shift(1)
#cambiando nombre de la columna, para combinar en el dataframe
ventaAyer.columns = ["Fecha", "PedidosAyerReales"]
ventaAyer.head()

,Fecha,PedidosAyerReales
0,2016-01-01,NaN
1,2016-01-02,74.0
2,2016-01-03,80.0
3,2016-01-04,79.0
4,2016-01-05,52.0


In [3]:
#incorporando la venta de ayer
df["PedidosAyerReales"] = ventaAyer["PedidosAyerReales"].copy()
df.head()

,Fecha,PedidosReales,SolerDiaPred,SolerNochePred,TotalesPred,difRealPred,difAbsRealPred,FechaYear,FechaMonth,FechaWeek,...,FechaDayofyear,FechaIs_month_end,FechaIs_month_start,FechaIs_quarter_end,FechaIs_quarter_start,FechaIs_year_end,FechaIs_year_start,FechaElapsed,PedidosRolling7d,PedidosAyerReales
0,2016-01-01,74,0,70,70,4,4,2016,1,53,...,1,0,1,0,1,0,1,1451606400,74,NaN
1,2016-01-02,80,0,79,79,1,1,2016,1,53,...,2,0,0,0,0,0,0,1451692800,154,74.0
2,2016-01-03,79,0,70,70,9,9,2016,1,53,...,3,0,0,0,0,0,0,1451779200,233,80.0
3,2016-01-04,52,0,52,52,0,0,2016,1,1,...,4,0,0,0,0,0,0,1451865600,285,79.0
4,2016-01-05,63,0,55,55,8,8,2016,1,1,...,5,0,0,0,0,0,0,1451952000,348,52.0


In [6]:
#columnsDrop = ["SolerDiaPred", "SolerNochePred", "TotalesPred", "difRealPred", "difAbsRealPred"]
#df = df.drop(columns=columnsDrop)
#df.info()

TypeError: drop() got an unexpected keyword argument 'columns'

In [7]:
cat_vars = ['FechaDayofweek', 'FechaYear', 'FechaMonth', 'FechaDay', 'FechaWeek', "FechaIs_month_end", "FechaIs_month_start", "FechaIs_quarter_end", 
            "FechaIs_quarter_start", "FechaIs_year_end", "FechaIs_year_start", "FechaDayofyear"]

contin_vars = ['PedidosRolling7d',"PedidosAyerReales"]

target = "PedidosReales"

n = len(df); n

846

In [8]:
joined = df[cat_vars + contin_vars+ [target, "Fecha"]].copy() #
joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 16 columns):
FechaDayofweek           846 non-null int64
FechaYear                846 non-null int64
FechaMonth               846 non-null int64
FechaDay                 846 non-null int64
FechaWeek                846 non-null int64
FechaIs_month_end        846 non-null int64
FechaIs_month_start      846 non-null int64
FechaIs_quarter_end      846 non-null int64
FechaIs_quarter_start    846 non-null int64
FechaIs_year_end         846 non-null int64
FechaIs_year_start       846 non-null int64
FechaDayofyear           846 non-null int64
PedidosRolling7d         846 non-null int64
PedidosAyerReales        845 non-null float64
PedidosReales            846 non-null int64
Fecha                    846 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(14)
memory usage: 105.8 KB


In [9]:
#TODO: VER sklearn.model_selection.TimeSeriesSplit

val_ratio = 0.15
test_ratio = 0.15
test_size = round(len(joined) * (1 - test_ratio) ) -1
train_size = round(test_size * (1 - val_ratio) ) -1
val_idx = list(range(train_size, test_size))
test_idx = list(range(test_size, len(joined)))

In [10]:
df_test = joined.iloc[test_idx].copy()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 718 to 845
Data columns (total 16 columns):
FechaDayofweek           128 non-null int64
FechaYear                128 non-null int64
FechaMonth               128 non-null int64
FechaDay                 128 non-null int64
FechaWeek                128 non-null int64
FechaIs_month_end        128 non-null int64
FechaIs_month_start      128 non-null int64
FechaIs_quarter_end      128 non-null int64
FechaIs_quarter_start    128 non-null int64
FechaIs_year_end         128 non-null int64
FechaIs_year_start       128 non-null int64
FechaDayofyear           128 non-null int64
PedidosRolling7d         128 non-null int64
PedidosAyerReales        128 non-null float64
PedidosReales            128 non-null int64
Fecha                    128 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(14)
memory usage: 17.0 KB


In [11]:
df_train = joined.drop(test_idx).copy()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 718 entries, 0 to 717
Data columns (total 16 columns):
FechaDayofweek           718 non-null int64
FechaYear                718 non-null int64
FechaMonth               718 non-null int64
FechaDay                 718 non-null int64
FechaWeek                718 non-null int64
FechaIs_month_end        718 non-null int64
FechaIs_month_start      718 non-null int64
FechaIs_quarter_end      718 non-null int64
FechaIs_quarter_start    718 non-null int64
FechaIs_year_end         718 non-null int64
FechaIs_year_start       718 non-null int64
FechaDayofyear           718 non-null int64
PedidosRolling7d         718 non-null int64
PedidosAyerReales        717 non-null float64
PedidosReales            718 non-null int64
Fecha                    718 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(14)
memory usage: 95.4 KB


In [12]:
df_train.head()

,FechaDayofweek,FechaYear,FechaMonth,FechaDay,FechaWeek,FechaIs_month_end,FechaIs_month_start,FechaIs_quarter_end,FechaIs_quarter_start,FechaIs_year_end,FechaIs_year_start,FechaDayofyear,PedidosRolling7d,PedidosAyerReales,PedidosReales,Fecha
0,4,2016,1,1,53,0,1,0,1,0,1,1,74,NaN,74,2016-01-01
1,5,2016,1,2,53,0,0,0,0,0,0,2,154,74.0,80,2016-01-02
2,6,2016,1,3,53,0,0,0,0,0,0,3,233,80.0,79,2016-01-03
3,0,2016,1,4,1,0,0,0,0,0,0,4,285,79.0,52,2016-01-04
4,1,2016,1,5,1,0,0,0,0,0,0,5,348,52.0,63,2016-01-05


In [13]:
X_train = df_train.iloc[:,:-2].values
y_train = df_train["PedidosReales"].values

In [14]:
X_test = df_test.iloc[:,:-2].values
y_test = df_test["PedidosReales"].values

In [15]:
print("(train,test)=> ({},{})".format(len(X_train), len(X_test)))

(train,test)=> (718,128)


In [16]:
#defino la métrica para optimizar
import math

def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    pct_var = (targ - y_pred)/targ
    return math.sqrt((pct_var**2).mean())

In [17]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=25, population_size=20, verbosity=2)#(generations=5, population_size=20, verbosity=2) #max_time_mins=20,
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
#tpot.export('tpot_model_pipeline.py')

Imputing missing values in feature set


Generation 1 - Current best internal CV score: -302.98792054061926


Generation 2 - Current best internal CV score: -302.98792054061926


Generation 3 - Current best internal CV score: -299.2125262164573


Generation 4 - Current best internal CV score: -299.2125262164573


...

Generation 22 - Current best internal CV score: -281.4851572945542


Generation 23 - Current best internal CV score: -279.05619654820714


Generation 24 - Current best internal CV score: -278.4695065557805


Generation 25 - Current best internal CV score: -278.4695065557805



Best pipeline: RidgeCV(LassoLarsCV(GradientBoostingRegressor(StandardScaler(Nystroem(GradientBoostingRegressor(input_matrix, alpha=0.95, learning_rate=0.01, loss=lad, max_depth=2, max_features=0.85, min_samples_leaf=6, min_samples_split=17, n_estimators=100, subsample=0.35), gamma=0.95, kernel=poly, n_components=8)), alpha=0.99, learning_rate=0.01, loss=quantile, max_depth=10, max_features=0.8, min_samples_leaf=7, min_samples_split=18, n_estimators=100, subsample=0.6), normalize=True))
-309.962871922


In [18]:
#best pipeline
print(tpot.fitted_pipeline_)
#list of all the pipeline used
#tpot.evaluated_individuals_

Pipeline(memory=None,
     steps=[('stackingestimator-1', StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.95, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='lad', max_depth=2,
             max_features=0.8500000000000001, max_leaf_nodes=None,
             min_impurity_decrease=0.0, m...None, fit_intercept=True, gcv_mode=None,
    normalize=False, scoring=None, store_cv_values=False))])


In [19]:
y_pred = tpot.predict(X_test)

In [20]:
exp_rmspe(y_pred, y_test)

0.1828583063950755

In [21]:
res = pd.DataFrame([y_pred,y_test]).T
res.head()

,0,1
0,139.360910,135.0
1,130.651559,96.0
2,68.687802,93.0
3,84.974741,91.0
4,94.840498,110.0


In [23]:
(res[0] - res[1]).abs().median()

10.196818401600257